In [91]:
import torch
import torchvision.datasets as dsets # 파이토치 중 이미지 처리에 필요한 기능들이 있음
import torch.nn as nn
import torchvision.transforms as transforms

In [92]:
'''if torch.cuda.is_available() == True:
    device = 'cuda'
else:
    device = 'cpu'
# torch.cuda.is_available() # GPU 를 사용하겠다.
device'''

"if torch.cuda.is_available() == True:\n    device = 'cuda'\nelse:\n    device = 'cpu'\n# torch.cuda.is_available() # GPU 를 사용하겠다.\ndevice"

In [93]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [94]:
learning_rate = 0.001 # 1e-3
training_epochs = 15
batch_size = 128

In [95]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환 (이게 없으면 numpy array로 가져옴)
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

In [96]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True) # 배치 마지막 깍두기 drop 여부 결정

In [97]:
len(mnist_train) % 128 # drop_last True 이면 나머지 96개는 버린다. 128 128 128 1 이면 안 돌아가는 경우도 있었다.

96

In [98]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # conv2d, maxpool2d, relu, linear

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # 필터 사이즈랑 개수가 다르기 때문에 conv를 2개로 나눠준다.
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.linear = nn.Linear(7 * 7 * 64, 10) # 출력되는 데이터의 개수 10개

        # 가중치 초기화를 추가할 수도 있다.

    def forward(self, x):
        # Conv1 block -> 14*14*32
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        # 데이터의 사이즈가 크면 여기에 batchnormalization 추가

        # Conv2 block -> 7*7*64
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)

        # Fully Connected Layer : 주로 분류 모델 끝단에 달려있다. 특정 작업 수행 단계
        # Flatten 같은 기능으로 펴줘야 됨, 우리는 일반적으로 view를 사용해서 형태 변형

        x.view(-1, 7*7*64) # 제일 끝에 있는 숫자를 7*7*64로 바꾸겠다.(128:배치, 64, 7, 7) -> (128, 3136) 이렇게 펴줘야 함.
                            # 따라서 배치는 알아서 하고 = -1, 뒤에는 전체의 곱을 넣어준다.
        x = self.linear(x)
        return x


In [99]:
model = CNN()
print(model)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear): Linear(in_features=3136, out_features=10, bias=True)
)


In [100]:
model.to(device)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear): Linear(in_features=3136, out_features=10, bias=True)
)

In [101]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [102]:
len(data_loader)

468

In [103]:
len(mnist_train) // 128

468

In [104]:
!pip install tqdm

In [105]:
for epoch in range(training_epochs):
    avg_loss = 0

    for X, y in tqdm.tqdm(data_loader):
        X = X.to(device)
        y = y.to(device)

        y_pred = model(X)

        loss = criterion(y_pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_loss += loss / len(data_loader)
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_loss))

  0%|          | 0/468 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (57344x7 and 3136x10)

In [ ]:
# with torch.no_grad() # 가중치를 업데이트 하지 않겠다. 추론과정에서 켜지면 안 되는 dropout 같은 기능도 다 꺼준다.


In [ ]:
mnist_test.test_data.shape # 뒤에 1이 들어가줘서 10000, 1, 28, 28 로 변환해야 한다. (view)

torch.Size([10000, 28, 28])

In [ ]:
# 학습을 진행하지 않을 것이므로 torch.no_grad() 사용
with torch.no_grad():
    # 테스트 데이터를 모델에 입력하기 위한 준비
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)  # 테스트 데이터셋의 크기를 맞추고, 연산을 위한 장치로 이동 (gpu로 이동)
    Y_test = mnist_test.test_labels.to(device)  # 테스트 데이터셋의 레이블을 연산을 위한 장치로 이동

    # 모델 예측 수행
    prediction = model(X_test)  # 테스트 데이터에 대해 모델이 예측한 결과값

    # 예측 결과와 실제 레이블 비교
    correct_prediction = torch.argmax(prediction, 1) == Y_test  # 예측된 클래스와 실제 레이블이 일치하는지 확인 (10000, 1)로 변환 과정 -> argmax

    # 정확도 계산
    accuracy = correct_prediction.float().mean()  # 정확도를 계산하기 위해 일치하는 예측의 평균을 구함
    print('Accuracy:', accuracy.item())  # 정확도를 출력